## 2.5 自动微分

求导是几乎所有深度学习优化算法的关键步骤。虽然求导的计算很简单，只需要一些基本的微积分。但对于复杂的模型，手工进行更新是一件很痛苦的事情。

深度学习框架通过自动计算导数，即自动微分(automatic differentiation)来加快求导。实际上，根据我们设计的模型，系统会构建一个计算图(computational graph)，来跟踪计算哪些数据通过哪些操作组合起来产生输出。自动微分使系统能够随后反向传播梯度。这里，反向传播(backpropagate)意味着跟着整个计算图，填充关于每个参数的偏导数。


### 2.5.1 一个简单的例子
作为一个演示例子，假设我们想对函数$y=2x^Tx$关于列向量$x$求导。首先，我们创建变量$x$并为其分配一个初始值。

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$x$的梯度之前，我们需要一个地方来存放梯度。重要的是，我们不会在每次对一个参数求导时都分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。注意，一个标量函数关于向量$x$的梯度是向量，并且与$x$具有相同的形状。

In [2]:
x.requires_grad_(True) #等价于x = torch.arange(4.0, requires_grad=True)
x.grad #默认值是None
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

$x$是一个长度为4的向量，计算$y$和$x$的点积，得到了我们赋值给$y$的标量输出。接下来，我们通过反向传播函数来自动计算$y$关于$x$每个分量的梯度，并打印这些梯度。

In [3]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$y = 2x^T x$关于$x$的梯度应为$4x$。让我们快速验证一下

In [4]:
x.grad == 4 * x

tensor([True, True, True, True])

现在让我们计算x的另一个函数


In [5]:
#在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])